### 1. 确认模型任务的原有行为

In [61]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

model.config.id2label = {0: "差评！", 1: "好评！"}
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

sen = "饭菜有些咸！"
pipe(sen)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': '好评！', 'score': 0.5472492575645447}]

### 2. 模型微调

#### 2.1 准备环境，导入相关包

In [41]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

#### 2.2 准备数据

- 加载数据

In [42]:
dataset = load_dataset("csv", data_files="./waimai_10k.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 11987
})

- 数据预处理

In [43]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_dataset = dataset.map(process_function, batched=True, remove_columns=dataset.column_names)
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 11987
})

- 分割数据集

In [44]:
tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.1)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10788
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1199
    })
})

#### 2.3 加载预训练模型

In [45]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

# if torch.cuda.is_available():
#     model = model.cuda()
#     print('Use cuda GPU')
# elif torch.backends.mps.is_available():
#     model = model.to('mps')
#     print('Use mps')
model.config

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertConfig {
  "_name_or_path": "hfl/rbt3",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 21128
}

#### 2.4 训练与评估

- 定义评估函数

In [46]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

- 定义训练参数

In [47]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               eval_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
evaluation_strategy=None,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=

In [48]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

- 执行训练

In [49]:
trainer.train()

  2%|▏         | 10/507 [00:03<02:18,  3.59it/s]

{'loss': 0.5992, 'grad_norm': 1.7134801149368286, 'learning_rate': 1.9605522682445763e-05, 'epoch': 0.06}


  4%|▍         | 20/507 [00:06<02:53,  2.81it/s]

{'loss': 0.5047, 'grad_norm': 2.125014543533325, 'learning_rate': 1.921104536489152e-05, 'epoch': 0.12}


  6%|▌         | 30/507 [00:09<02:42,  2.93it/s]

...

100%|██████████| 507/507 [03:03<00:00,  3.40it/s]

{'eval_loss': 0.23135297000408173, 'eval_accuracy': 0.9174311926605505, 'eval_f1': 0.8748419721871049, 'eval_runtime': 2.3411, 'eval_samples_per_second': 512.143, 'eval_steps_per_second': 4.271, 'epoch': 3.0}


100%|██████████| 507/507 [03:04<00:00,  2.75it/s]

{'train_runtime': 184.3464, 'train_samples_per_second': 175.561, 'train_steps_per_second': 2.75, 'train_loss': 0.27392145938421847, 'epoch': 3.0}


TrainOutput(global_step=507, training_loss=0.27392145938421847, metrics={'train_runtime': 184.3464, 'train_samples_per_second': 175.561, 'train_steps_per_second': 2.75, 'total_flos': 471647796782640.0, 'train_loss': 0.27392145938421847, 'epoch': 3.0})

- 评估

In [51]:
trainer.evaluate()

100%|██████████| 10/10 [00:01<00:00,  5.10it/s]


{'eval_loss': 0.23134386539459229,
 'eval_accuracy': 0.9182652210175146,
 'eval_f1': 0.8759493670886076,
 'eval_runtime': 2.2119,
 'eval_samples_per_second': 542.056,
 'eval_steps_per_second': 4.521,
 'epoch': 3.0}

#### 2.5 模型预测

In [52]:
trainer.predict(tokenized_datasets["test"])

100%|██████████| 10/10 [00:01<00:00,  5.14it/s]


PredictionOutput(predictions=array([[ 1.2873403, -1.068345 ],
       [-2.1651404,  2.165618 ],
       [ 1.9800029, -2.0699615],
       ...,
       [-0.5200064,  0.4800528],
       [ 1.6012837, -1.5406739],
       [ 1.6972494, -2.3993273]], dtype=float32), label_ids=array([0, 1, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.23134386539459229, 'test_accuracy': 0.9182652210175146, 'test_f1': 0.8759493670886076, 'test_runtime': 2.1935, 'test_samples_per_second': 546.621, 'test_steps_per_second': 4.559})

In [53]:
from transformers import pipeline

id2_label = {0: "差评！", 1: "好评！"}
model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

sen = "饭菜有些咸！"
pipe(sen)

[{'label': '差评！', 'score': 0.9463842511177063}]

#### 2.6 微调模型保存

- 保存到本地

In [56]:
local_model_path = './my-awesome-model'
model.save_pretrained(local_model_path)
tokenizer.save_pretrained(local_model_path)

('./my-awesome-model/tokenizer_config.json',
 './my-awesome-model/special_tokens_map.json',
 './my-awesome-model/vocab.txt',
 './my-awesome-model/added_tokens.json',
 './my-awesome-model/tokenizer.json')

- 从本地加载和预测

In [57]:
model = AutoModelForSequenceClassification.from_pretrained(local_model_path)
model.config.id2label = id2_label

tokenizer = AutoTokenizer.from_pretrained(local_model_path)
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

sen = "饭菜有些咸！"
pipe(sen)

[{'label': '差评！', 'score': 0.9463842511177063}]